# Aprendizado Semi-Supervisionado

## Bibliotecas

In [1]:
from sklearn.svm import SVC
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd

## Base de dados

In [5]:
df = pd.read_excel('BD_TCC.xlsx', sheet_name= "BD_Classificados")
df

,Chave,Loja,Familia,Tipo de Consumo,Lead Time,Consumo Médio Anual,Preço Médio,ABC,ABC Analista
0,"1MAT ELETRICOCONSUMO MEDIO409258,7433",1,MAT ELETRICO,CONSUMO MEDIO,40,9.000000,258.7433,A,NaN
1,"10MAT ELETRICOCONSUMO MEDIO3040,33333333333333...",10,MAT ELETRICO,CONSUMO MEDIO,30,40.333333,30.7300,B,NaN
2,"1MAT ELETRICOCONSUMO MEDIO3056,333333333333330,73",1,MAT ELETRICO,CONSUMO MEDIO,30,56.333333,30.7300,A,NaN
3,"10MAT ELETRICOCONSUMO MEDIO305,666666666666675...",10,MAT ELETRICO,CONSUMO MEDIO,30,5.666667,58.8400,B,A
4,"5MAT ELETRICOCONSUMO MEDIO3015,333333333333358,84",5,MAT ELETRICO,CONSUMO MEDIO,30,15.333333,58.8400,B,NaN
...,...,...,...,...,...,...,...,...,...
6525,"5IMPORTADOSCONSUMO MEDIO993769,83",5,IMPORTADOS,CONSUMO MEDIO,99,3.000000,769.8300,A,NaN
6526,"1MAT ELETRICOCONSUMO MEDIO402124,96",1,MAT ELETRICO,CONSUMO MEDIO,40,2.000000,124.9600,C,NaN
6527,"1VEDACAOCONSUMO MEDIO40412,52",1,VEDACAO,CONSUMO MEDIO,40,4.000000,12.5200,B,NaN
6528,"5PARAFUSOSCONSUMO MEDIO30309,6666666666671,0733",5,PARAFUSOS,CONSUMO MEDIO,30,309.666667,1.0733,B,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6530 entries, 0 to 6529
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Chave                6530 non-null   object 
 1   Loja                 6530 non-null   int64  
 2   Familia              6530 non-null   object 
 3   Tipo de Consumo      6530 non-null   object 
 4   Lead Time            6530 non-null   int64  
 5   Consumo Médio Anual  6530 non-null   float64
 6   Preço Médio          6530 non-null   float64
 7   ABC                  6530 non-null   object 
 8   ABC Analista         392 non-null    object 
dtypes: float64(2), int64(2), object(5)
memory usage: 459.3+ KB


## Definição das variáveis

In [7]:
# Suponha que df seja seu DataFrame completo com 6000 linhas
# E que 'classe_abc' é a coluna com os rótulos dos 400 dados rotulados, e NaN para os outros

# Separar variáveis
cat_features = ['Familia'] #Categóricas
num_features = ['Consumo Médio Anual', 'Preço Médio'] #Numéricas

In [10]:
X = df[cat_features + num_features]

In [11]:
# Criar vetor de rótulos: -1 para não rotulados
y = df['ABC Analista'].copy()
y = y.fillna(-1)

## Pré-processamento

In [13]:
# Pré-processamento
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features),
    ('num', StandardScaler(), num_features)
])

In [14]:
# Classificador base
base_classificador_svm = SVC(probability=True, C= 1.0, kernel= 'rbf', tol= 0.0001, random_state=42)

In [15]:
# Modelo semi-supervisionado
self_training = SelfTrainingClassifier(
    base_classificador_svm, 
    threshold=0.8, 
    verbose=True
)

## Criação e treinamento do pipeline

In [16]:
# Pipeline completo
pipeline = Pipeline([
    ('preproc', preprocessor),
    ('self_train', self_training)
])

In [17]:
# Treinamento
print("Iniciando o treinamento do modelo semi-supervisionado com SVM...")
# Treina o modelo com todos os dados (rotulados e não rotulados)
pipeline.fit(X, y)
print("---------------")
print("Treinamento concluído!")

Iniciando o treinamento do modelo semi-supervisionado com SVM...
End of iteration 1, added 4733 new labels.
End of iteration 2, added 929 new labels.
End of iteration 3, added 429 new labels.
End of iteration 4, added 23 new labels.
End of iteration 5, added 5 new labels.
---------------
Treinamento concluído!


## Previsão e salvamento dos dados

In [18]:
print("Realizando a previsão para todos os materiais...")
y_pred = pipeline.predict(X)

Realizando a previsão para todos os materiais...


In [20]:
# Adiciona os resultados como uma nova coluna no DataFrame original
df['ABC_SVM'] = y_pred
# Salva o DataFrame completo com as previsões em um novo arquivo Excel
salvar = 'Resultado_final_com_previsao_svm.xlsx'
df.to_excel(salvar, index=False)